### [DART] 단일회사 전체 재무제표
* https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS003&apiId=2019020

In [3]:
import requests

# 상장기업 재무정보 - 단일회사 전체 재무제표
url = 'https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json'
api_key = 'f2e08d4ed3de0ba3d5cbf59c04c223e02b1751a2'
corp_code = '01208885'
fs_div = 'OFS'      # CFS:연결재무제표, OFS:재무제표
fyear = 2022

params = {
    'corp_code': corp_code,
    'bsns_year': fyear,         # 사업연도
    'reprt_code': 11011,        # 사업보고서
    'fs_div': fs_div,           # 개별기업 재무제표
    'crtfc_key': api_key,
}

response = requests.get(url, params=params)
if response.status_code != 200:     # URL GET '200 정상'
    print("URL GET Error")

- BS : 재무상태표
- IS : 손익계산서
- CIS : 포괄손익계산서
- CF : 현금흐름표
- SCE : 자본변동표

In [4]:
# 데이터프레임 형태로 변환, 데이터 타입 변환
import pandas as pd
df_fi = pd.json_normalize(response.json()['list'])
df_fi['thstrm_amount'].fillna(0)
df_fi['frmtrm_amount'].fillna(0)
df_fi['bfefrmtrm_amount'].fillna(0)
df_fi['thstrm_add_amount'].fillna(0)
df_fi = df_fi.astype({'ord' : 'int'})

#### 포괄손익계산서

In [5]:
sj_div = "CIS"  # 포괄손익계산서
df_cis = df_fi.query('sj_div == @sj_div').sort_values('ord')

> 항목: XBRL 표준계정ID
- 매출액 ifrs-full_Revenue
- 영업이익 dart_OperatingIncomeLoss
- 비영업이익
- 연구개발비
- 광고선전비
- 이자비용
- EBIT

In [6]:
acct_id = ['ifrs-full_Revenue', 'dart_OperatingIncomeLoss']
df_cis.query('account_id in @acct_id')[['account_id', 'account_nm', 'thstrm_amount', 'currency']]          # 당기금액

,account_id,account_nm,thstrm_amount,currency
49,ifrs-full_Revenue,매출액,2304288140077,KRW
58,dart_OperatingIncomeLoss,영업이익,25053659119,KRW


#### 재무상태표

In [7]:
sj_div = "BS"  # 재무상태표
df_bs = df_fi.query('sj_div == @sj_div').sort_values('ord')

> 항목 XBRL 표준계정ID
- 총자산 ifrs-full_Assets
- 총부채 ifrs-full_Liabilities

In [8]:

acct_id = ['ifrs-full_Assets',
           'ifrs-full_Liabilities']
df_bs.query('account_id in @acct_id')[['account_id', 'account_nm', 'thstrm_amount', 'currency']]          # 당기금액
# df_bs[df_bs['account_nm'].str.contains('자산')]

,account_id,account_nm,thstrm_amount,currency
22,ifrs-full_Assets,자산총계,1064615087230,KRW
40,ifrs-full_Liabilities,부채총계,672093137209,KRW


#### 현금흐름표

In [9]:
sj_div = "CF"  # 현금흐름표
df_cf = df_fi.query('sj_div == @sj_div').sort_values('ord')

> 항목 XBRL 표준계정ID
- 영업활동현금흐름 ifrs-full_CashFlowsFromUsedInOperatingActivities
- 영업으로부터 창출된 현금흐름 dart_NetCashflowsFromUsedInOperations

In [10]:
acct_id = ['ifrs-full_CashFlowsFromUsedInOperatingActivities',
           'dart_NetCashflowsFromUsedInOperations']
df_cf.query('account_id in @acct_id')[['account_id', 'account_nm', 'thstrm_amount', 'currency']]          # 당기금액

,account_id,account_nm,thstrm_amount,currency
73,ifrs-full_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,52246134856,KRW
74,dart_NetCashflowsFromUsedInOperations,영업으로부터 창출된 현금흐름,60515784908,KRW
